# Custom loader

## Imports

In [ ]:
import numpy as np
import photon_tools as pt

from photon_tools.model import PhotonData, PhotonDataset

## Define a custom loader

**Why this is useful**
- Everyone can extend photon-tools in their own scripts/notebooks
- No fork needed; no changes to the package source

In [ ]:
def my_dummy_loader(path, *, timing_resolution=1e-6, **kwargs):
    """
    Example custom loader.

    This simulates loading a file format that photon-tools does not know.
    The loader returns a PhotonDataset in the standard in-memory format.
    """

    # Fake photon stream (ticks)
    ts = np.array([0, 1, 2, 10, 11, 12, 50, 55], dtype=np.int64)

    # Fake detector assignment (two detectors)
    det = np.array([0, 1, 0, 1, 0, 1, 0, 1], dtype=np.int16)

    # Fake nanotimes (e.g. TCSPC bins or phase within a sync cycle)
    # Must have the SAME length as timestamps
    nt = np.array([5, 12, 7, 20, 18, 3, 9, 15], dtype=np.int64)

    photons = PhotonData(
        timestamps=ts,
        detectors=det,
        nanotimes=nt,
        timing_resolution=float(timing_resolution),
    )

    meta = {
        "format": "dummy",
        "note": "example custom loader with nanotimes",
        "nanotimes_kind": "example",  # optional but informative
    }

    raw = {
        "path": str(path),
        "kwargs": kwargs,
    }

    return PhotonDataset(photons=photons, meta=meta, raw=raw, source=str(path))

## Register the loader at runtime

In [ ]:
pt.register_loader(".dummy", loader=my_dummy_loader, overwrite=True)

## Use pt.load() with the new format

In [ ]:
ds = pt.load("test.dummy", timing_resolution=5e-9, my_option="hello")
ds.meta, ds.photons.timestamps[:5], ds.photons.detectors[:5]

## Preview the result

In [ ]:
fig = pt.preview(
    ds,
    bin_width_ms=1,
    detector_labels={0: "custom0", 1: "custom1"},
)